Testing Jun2 

Base test for BRK.B 

In [1]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np 
from IPython.display import display


In [2]:
#Dataset 
file_path = '/Users/schoudhry/Desktop/shashy/traderAuto/BRK.B(jan9.2024).csv'
df = pd.read_csv(file_path)

# Clean up column names and format date
df.columns = df.columns.str.strip()
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

# Display the entire DataFrame (remove row limit)
pd.set_option('display.max_rows', None)
#display(df)

In [3]:
#Candlestick for BRK.B 

fig = go.Figure(data=[go.Candlestick(
    x=df['Date'],
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name='BRK.B'
)])

fig.update_layout(
    title='BRK.B Daily Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price ($)',
    xaxis_rangeslider_visible=False,
    template='plotly_white',
    height=600
)

fig.show()
    

Function for trade val call:
    function takes in data from the observed DataFrame (every day open/sell)
    from here check if daily values are given through data frame 
    use candle stick to verify
    use one of the mathmatical models to calculate the ideal share price(s)
    then compare against a simple human buy/sell rate 
    those values should get you a simple comparsion dataframe
    the data frame should show the stock buy/sell and the profit margin between both strats
    run the eval algo over a 1yr historical data 
    then will run the algo using irl data for 6 months before anything else 



Test Method: 
Method 1 
Using SciKit Learn to find the best price 

use the Discounted Cash Flow valuation method 
approximation mehtod based on the its future cash flow 
base the algorithm on lower and upper echelon 
based on buying and selling expenditures 
from there base it only on short term data
from the short term data extract best cash flow processing 
should be scaleable from a 10000 buy in at a 100/125 sell/buy rate 
down to a 1000 buy in at a sell rate of 10/12.5 sell/buy rate 



Set a call method to get a 
track into a csv for each stock 
every buy/sell and impact to cash vs stock 

need the algorithm to run on the data from every day 


TO CHECK once BRK.B is running relatively optimized 

QQQ -> Invesco 
QQQm -> INvesco Nasdaq 100 etf 
IGM -> iShares Expended Tech Sector ETF
VTI -> vanguar dtotal stock market index fund ETF 
IYW -> iShares US tech ETF 
WMT -> Walmart 
COST -> Costco 
EL -> Estee Lauder 
ALLE -> Allegion 
RIVN -> 
CVX -> Chevron 
BTC -> bitcoin (human: buy under 93.5k sell above 100.5k)

Alert based buy ins:
AEP -> calculate buy in price/sell based on 1yr highs and lows 
human: buy in 96.5 sell at 101.5 





Look into trading options 

Ideal algorithm should aim for a win rate of 35 -> 50 % 
know when to cut the trade

Penny Stock Algo
Outstanding Shares(OS) = all the shsares company has made 
Float = the amnt of shares the public can trade 

Smaller Float means bigger moves 
if fewer shares exist then the price is more voilatile

The aim is to find low float stocks 
under 20mil,10mil,5mil 

Volume will be the watch criteria 
high volume means that that a change is happening 
if normal is 100,000 shares per day and then its doing 5 million 
float roation ^
the aim is low float and abromal volume 
wait for the breakout 

Check for catalyst(s)
Zkin (human buy at sub 1.00 sell at 2.50 -> 3.00)

look into using the finviz screener 

augmentation mechanic 
sentatiment data + stock analysis 



HYPR
LYRA
APLD
BMGL
OGEN
BNAI



In [ ]:

# === Constants ===
initial_investment = 1000.0
initial_price = 468.5
buy_amount = 125.0
sell_amount = 100.0
latest_close = df.iloc[-1]['Close']

# === Strategy Function ===
def simulate_strategy_open_price(df, buy_thresh, sell_thresh, label):
    cash = 0.0
    shares = initial_investment / initial_price
    trades = 0
    log = []

    for _, row in df.iterrows():
        date = row['Date']
        open_price = row['Open']
        close_price = row['Close']
        action = "Hold"
        shares_traded = 0.0
        cash_change = 0.0

        if open_price <= buy_thresh:
            shares_bought = buy_amount / open_price
            shares += shares_bought
            cash -= buy_amount
            trades += 1
            action = "Buy"
            shares_traded = shares_bought
            cash_change = -buy_amount

        elif open_price >= sell_thresh:
            shares_to_sell = sell_amount / open_price
            if shares >= shares_to_sell:
                shares -= shares_to_sell
                cash += sell_amount
                trades += 1
                action = "Sell"
                shares_traded = -shares_to_sell
                cash_change = sell_amount

        log.append({
            "Date": date,
            "Strategy": label,
            "Action": action,
            "Price": open_price,
            "Shares Traded": shares_traded,
            "Cash Change": cash_change,
            "Shares Held": shares,
            "Cash": cash,
            "Portfolio Value": cash + shares * close_price
        })

    final_value = cash + shares * latest_close
    net_gain = final_value - initial_investment
    action_counts = pd.DataFrame(log)["Action"].value_counts().to_dict()

    return pd.DataFrame(log), final_value, net_gain, trades, action_counts

# === Run Human Strategy ===
human_log, human_val, human_gain, human_trades, human_counts = simulate_strategy_open_price(
    df, buy_thresh=498.5, sell_thresh=500.5, label="Human Strategy"
)

# === Run AI Optimization ===
best_val = 0
best_gain = 0
best_buy = None
best_sell = None
best_log = None
best_counts = {}

for buy in np.arange(480, 500.5, 0.5):
    for sell in np.arange(500, 520.5, 0.5):
        temp_log, val, gain, trades, counts = simulate_strategy_open_price(df, buy, sell, "AI Strategy")
        if val > best_val:
            best_val = val
            best_gain = gain
            best_buy = buy
            best_sell = sell
            best_log = temp_log
            best_counts = counts

# === Final summary table ===
summary = pd.DataFrame([
    {
        "Strategy": "Human Strategy",
        "Buy Count": human_counts.get("Buy", 0),
        "Sell Count": human_counts.get("Sell", 0),
        "Hold Count": human_counts.get("Hold", 0),
        "Final Value": round(human_val, 2),
        "Net Gain": round(human_gain, 2),
        "Return (%)": round((human_gain / initial_investment) * 100, 2)
    },
    {
        "Strategy": f"AI Strategy (Buy ≤ {best_buy}, Sell ≥ {best_sell})",
        "Buy Count": best_counts.get("Buy", 0),
        "Sell Count": best_counts.get("Sell", 0),
        "Hold Count": best_counts.get("Hold", 0),
        "Final Value": round(best_val, 2),
        "Net Gain": round(best_gain, 2),
        "Return (%)": round((best_gain / initial_investment) * 100, 2)
    }
])

display(summary)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Load and prepare dataset
file_path = "/Users/schoudhry/Desktop/shashy/traderAuto/BRK.B(jan9.2024).csv"  # CHANGE THIS if needed
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)

# Feature Engineering
df['Prev_Close'] = df['Close'].shift(1)
df['Prev_Open'] = df['Open'].shift(1)
df['3Day_MA'] = df['Close'].rolling(window=3).mean()
df['5Day_MA'] = df['Close'].rolling(window=5).mean()
df['Return'] = df['Close'].pct_change()
df['Volatility'] = df['Close'].rolling(window=5).std()

# Target: Profit by buying at open and selling at close
df['Profit'] = df['Close'] - df['Open']

# Drop NA rows
df = df.dropna().reset_index(drop=True)

# Features and Target
features = ['Prev_Close', 'Prev_Open', '3Day_MA', '5Day_MA', 'Return', 'Volatility']
X = df[features]
y = df['Profit']

# Train RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# Predict expected profit
df['Predicted_Profit'] = model.predict(X)

# Grid search for optimal thresholds
thresholds = np.arange(0.05, 2.0, 0.05)
best_buy_thresh = None
best_sell_thresh = None
best_value = -np.inf

for buy_thresh in thresholds:
    for sell_thresh in thresholds:
        df['Action'] = 'Hold'
        df.loc[df['Predicted_Profit'] <= -buy_thresh, 'Action'] = 'Buy'
        df.loc[df['Predicted_Profit'] >= sell_thresh, 'Action'] = 'Sell'

        cash = 0
        shares = 1000 / df.loc[0, 'Open']  # Starting with $1000 initial buy
        for i in range(1, len(df)):
            price = df.loc[i, 'Open']
            if df.loc[i, 'Action'] == 'Buy':
                buy_amt = 100
                if cash >= buy_amt or cash == 0:
                    shares += buy_amt / price
                    cash -= buy_amt
            elif df.loc[i, 'Action'] == 'Sell' and shares > 0:
                sell_amt = 100
                shares_to_sell = sell_amt / price
                if shares >= shares_to_sell:
                    shares -= shares_to_sell
                    cash += sell_amt

        final_value = cash + shares * df.iloc[-1]['Close']
        if final_value > best_value:
            best_value = final_value
            best_buy_thresh = buy_thresh
            best_sell_thresh = sell_thresh

# Print optimal strategy
print("----- OPTIMAL AI STRATEGY -----")
print(f"Buy if predicted profit <= -${round(best_buy_thresh, 2)}")
print(f"Sell if predicted profit >= ${round(best_sell_thresh, 2)}")
print(f"Final Portfolio Value: ${round(best_value, 2)}")


----- OPTIMAL AI STRATEGY -----
Buy if predicted profit <= -$1.5
Sell if predicted profit >= $1.95
Final Portfolio Value: $1166.1
